## Chapter 13: Authentication and Authorization

Now that you can build feature‑rich applications with data access, APIs, logging, and configuration, it’s time to secure them. Most applications need to know who the user is (**authentication**) and what they are allowed to do (**authorization**). ASP.NET Core provides a flexible, claims‑based infrastructure that works for both traditional web apps (with cookies) and modern APIs (with JWT tokens). In this chapter, you’ll learn the fundamentals of authentication and authorization, how to implement ASP.NET Core Identity for user management, how to secure MVC applications with cookies, how to protect Web APIs with JWT, and how to use policies and roles to control access. By the end, you’ll be able to build secure applications that protect data and enforce business rules.

### 13.1 Understanding Identity: Claims, Principals, and Identities

Before diving into code, you need to understand the core concepts of the ASP.NET Core security model.

#### Authentication vs. Authorization

- **Authentication** is the process of verifying a user’s identity. It answers the question “Who are you?” Common mechanisms: username/password, social logins, tokens.
- **Authorization** is the process of determining what an authenticated user is allowed to do. It answers “What can you do?”

#### Claims‑Based Identity

In ASP.NET Core, a user’s identity is represented as a set of **claims**. A claim is a piece of information about the user, such as their name, email address, role, or a custom permission. Claims are issued by a trusted party (an **identity provider**) and are packaged into an **identity**. A user may have multiple identities (e.g., from different login providers).

- **Claim**: A key‑value pair with an issuer. Example: `ClaimTypes.Email = "user@example.com"`.
- **ClaimsIdentity**: A collection of claims, representing one authenticated identity. It has a name and authentication type.
- **ClaimsPrincipal**: Represents the user, containing one or more `ClaimsIdentity` objects. In a web app, `HttpContext.User` is a `ClaimsPrincipal`.

After successful authentication, the runtime creates a `ClaimsPrincipal` and attaches it to the current request. You can access it anywhere via `HttpContext.User` or in controllers via the `User` property.

#### How It Works in Practice

When a user logs in:
1. You verify their credentials.
2. You create a `ClaimsIdentity` with relevant claims (e.g., user ID, name, roles).
3. You sign in by creating a cookie (or a JWT token) that contains this identity.
4. On subsequent requests, the cookie/token is automatically read and the `User` property is populated.

Authorization checks (like `[Authorize]`) examine the `User` property to decide whether to grant access.

### 13.2 Implementing ASP.NET Core Identity (User Registration, Login)

**ASP.NET Core Identity** is a full‑featured membership system that handles user registration, login, password hashing, role management, and more. It works with Entity Framework Core to store users in a database.

#### Setting Up Identity

1. **Install the Identity packages** (already included in some templates):
   ```bash
   dotnet add package Microsoft.AspNetCore.Identity.EntityFrameworkCore
   dotnet add package Microsoft.AspNetCore.Identity.UI
   ```

2. **Add a User class** that inherits from `IdentityUser`. You can add custom properties if needed.

   ```csharp
   using Microsoft.AspNetCore.Identity;

   public class ApplicationUser : IdentityUser
   {
       // Add custom properties here
       public string FullName { get; set; }
   }
   ```

3. **Update your `DbContext`** to inherit from `IdentityDbContext<ApplicationUser>`.

   ```csharp
   public class AppDbContext : IdentityDbContext<ApplicationUser>
   {
       public AppDbContext(DbContextOptions<AppDbContext> options)
           : base(options) { }

       // Your other DbSets (Products, Categories, etc.)
   }
   ```

4. **Add Identity services** in `Program.cs`:

   ```csharp
   builder.Services.AddDbContext<AppDbContext>(options =>
       options.UseSqlServer(builder.Configuration.GetConnectionString("DefaultConnection")));

   builder.Services.AddIdentity<ApplicationUser, IdentityRole>()
       .AddEntityFrameworkStores<AppDbContext>()
       .AddDefaultTokenProviders();

   // If you want the default UI (login, register pages), add:
   builder.Services.AddRazorPages(); // needed for Identity UI
   ```

5. **Add authentication middleware** in the request pipeline (order matters!):

   ```csharp
   app.UseAuthentication(); // Must come before UseAuthorization
   app.UseAuthorization();
   ```

6. **Create and run migrations** to create Identity tables:
   ```bash
   dotnet ef migrations add AddIdentityTables
   dotnet ef database update
   ```

#### Using Identity in Controllers and Views

Identity provides a set of Razor Pages for registration, login, etc., if you include the UI. You can scaffold them to customize.

To access the current user in a controller, use the `User` property:

```csharp
public class HomeController : Controller
{
    public IActionResult Index()
    {
        if (User.Identity.IsAuthenticated)
        {
            var userName = User.Identity.Name;
            var email = User.FindFirstValue(ClaimTypes.Email);
            // ...
        }
        return View();
    }
}
```

To protect an action, add the `[Authorize]` attribute:

```csharp
[Authorize]
public IActionResult Profile()
{
    return View();
}
```

#### Registering a User Manually

If you need custom registration logic, you can inject `UserManager<ApplicationUser>`:

```csharp
public class AccountController : Controller
{
    private readonly UserManager<ApplicationUser> _userManager;
    private readonly SignInManager<ApplicationUser> _signInManager;

    public AccountController(UserManager<ApplicationUser> userManager, SignInManager<ApplicationUser> signInManager)
    {
        _userManager = userManager;
        _signInManager = signInManager;
    }

    [HttpPost]
    public async Task<IActionResult> Register(RegisterViewModel model)
    {
        if (ModelState.IsValid)
        {
            var user = new ApplicationUser
            {
                UserName = model.Email,
                Email = model.Email,
                FullName = model.FullName
            };
            var result = await _userManager.CreateAsync(user, model.Password);
            if (result.Succeeded)
            {
                // Optionally sign the user in
                await _signInManager.SignInAsync(user, isPersistent: false);
                return RedirectToAction("Index", "Home");
            }
            foreach (var error in result.Errors)
            {
                ModelState.AddModelError(string.Empty, error.Description);
            }
        }
        return View(model);
    }
}
```

#### Logging In and Out

```csharp
[HttpPost]
public async Task<IActionResult> Login(LoginViewModel model)
{
    if (ModelState.IsValid)
    {
        var result = await _signInManager.PasswordSignInAsync(model.Email, model.Password, model.RememberMe, lockoutOnFailure: false);
        if (result.Succeeded)
        {
            return RedirectToLocal(returnUrl);
        }
        if (result.IsLockedOut)
        {
            return View("Lockout");
        }
        else
        {
            ModelState.AddModelError(string.Empty, "Invalid login attempt.");
        }
    }
    return View(model);
}

[HttpPost]
public async Task<IActionResult> Logout()
{
    await _signInManager.SignOutAsync();
    return RedirectToAction("Index", "Home");
}
```

### 13.3 Cookie Authentication (for MVC apps)

Even without Identity, you can use cookie authentication directly. This is useful when you have your own user store or want a simpler setup.

#### Setting Up Cookie Authentication

```csharp
builder.Services.AddAuthentication(CookieAuthenticationDefaults.AuthenticationScheme)
    .AddCookie(options =>
    {
        options.LoginPath = "/Account/Login";
        options.LogoutPath = "/Account/Logout";
        options.AccessDeniedPath = "/Account/AccessDenied";
        options.ExpireTimeSpan = TimeSpan.FromDays(7);
        options.SlidingExpiration = true;
    });
```

In the same `Program.cs`, you still need `app.UseAuthentication()` and `app.UseAuthorization()`.

#### Signing In with Cookies

When a user logs in, you create a `ClaimsPrincipal` and sign in:

```csharp
[HttpPost]
public async Task<IActionResult> Login(string username, string password)
{
    // Validate credentials against your own logic
    if (IsValidUser(username, password))
    {
        var claims = new List<Claim>
        {
            new Claim(ClaimTypes.Name, username),
            new Claim(ClaimTypes.Email, $"{username}@example.com"),
            // Add custom claims
        };

        var claimsIdentity = new ClaimsIdentity(claims, CookieAuthenticationDefaults.AuthenticationScheme);
        var authProperties = new AuthenticationProperties
        {
            IsPersistent = true, // remember me
            ExpiresUtc = DateTimeOffset.UtcNow.AddDays(7)
        };

        await HttpContext.SignInAsync(
            CookieAuthenticationDefaults.AuthenticationScheme,
            new ClaimsPrincipal(claimsIdentity),
            authProperties);

        return RedirectToAction("Index", "Home");
    }
    ModelState.AddModelError("", "Invalid login attempt.");
    return View();
}
```

#### Signing Out

```csharp
[HttpPost]
public async Task<IActionResult> Logout()
{
    await HttpContext.SignOutAsync(CookieAuthenticationDefaults.AuthenticationScheme);
    return RedirectToAction("Index", "Home");
}
```

### 13.4 JWT (JSON Web Token) Authentication (for Web APIs)

For Web APIs and Single Page Applications (SPAs), JWT is the standard. A JWT token is a self‑contained token that includes claims and is digitally signed. The client sends the token in the `Authorization` header.

#### Setting Up JWT Authentication

Install the package:

```bash
dotnet add package Microsoft.AspNetCore.Authentication.JwtBearer
```

Add and configure JWT authentication in `Program.cs`:

```csharp
using Microsoft.AspNetCore.Authentication.JwtBearer;
using Microsoft.IdentityModel.Tokens;
using System.Text;

// ...

var jwtSettings = builder.Configuration.GetSection("JwtSettings");
var secretKey = Encoding.UTF8.GetBytes(jwtSettings["Secret"]);

builder.Services.AddAuthentication(options =>
{
    options.DefaultAuthenticateScheme = JwtBearerDefaults.AuthenticationScheme;
    options.DefaultChallengeScheme = JwtBearerDefaults.AuthenticationScheme;
})
.AddJwtBearer(options =>
{
    options.RequireHttpsMetadata = false; // set to true in production
    options.SaveToken = true;
    options.TokenValidationParameters = new TokenValidationParameters
    {
        ValidateIssuerSigningKey = true,
        IssuerSigningKey = new SymmetricSecurityKey(secretKey),
        ValidateIssuer = true,
        ValidIssuer = jwtSettings["Issuer"],
        ValidateAudience = true,
        ValidAudience = jwtSettings["Audience"],
        ValidateLifetime = true,
        ClockSkew = TimeSpan.Zero
    };
});
```

Add configuration in `appsettings.json`:

```json
"JwtSettings": {
  "Secret": "YourSuperSecretKeyHereAtLeast32CharsLong",
  "Issuer": "MyApp",
  "Audience": "MyAppClients"
}
```

**Important:** Never hardcode secrets; use environment variables or Azure Key Vault.

#### Generating a JWT Token

In your login endpoint, validate credentials and issue a token:

```csharp
[HttpPost("login")]
public IActionResult Login([FromBody] LoginModel model)
{
    if (IsValidUser(model.Username, model.Password))
    {
        var token = GenerateJwtToken(model.Username);
        return Ok(new { token });
    }
    return Unauthorized();
}

private string GenerateJwtToken(string username)
{
    var claims = new[]
    {
        new Claim(JwtRegisteredClaimNames.Sub, username),
        new Claim(JwtRegisteredClaimNames.Jti, Guid.NewGuid().ToString()),
        new Claim(ClaimTypes.Name, username),
        // Add roles or other claims
    };

    var key = new SymmetricSecurityKey(Encoding.UTF8.GetBytes(_configuration["JwtSettings:Secret"]));
    var creds = new SigningCredentials(key, SecurityAlgorithms.HmacSha256);

    var token = new JwtSecurityToken(
        issuer: _configuration["JwtSettings:Issuer"],
        audience: _configuration["JwtSettings:Audience"],
        claims: claims,
        expires: DateTime.Now.AddHours(1),
        signingCredentials: creds);

    return new JwtSecurityTokenHandler().WriteToken(token);
}
```

#### Using the Token in Client Requests

The client must include the token in the `Authorization` header:

```
Authorization: Bearer <your-token>
```

In your API controllers, you can use `[Authorize]` to protect endpoints. The framework will automatically validate the token and populate `User` with the claims.

### 13.5 Authorization: `[Authorize]` and `[AllowAnonymous]`

#### Basic Usage

- `[Authorize]` – allows only authenticated users.
- `[AllowAnonymous]` – overrides `[Authorize]` at a higher level (e.g., on a controller, you can allow anonymous access to a specific action).

```csharp
[Authorize]
public class AccountController : Controller
{
    [AllowAnonymous]
    public IActionResult Login() => View();

    [Authorize] // redundant but explicit
    public IActionResult Profile() => View();
}
```

#### Restricting by Roles

If you have roles assigned to users, you can restrict access with the `Roles` property:

```csharp
[Authorize(Roles = "Admin")]
public IActionResult AdminPanel() => View();

[Authorize(Roles = "Admin,Manager")]
public IActionResult Reports() => View();
```

#### Using Policies

Policies are more flexible and allow you to define complex requirements (e.g., “age over 18” or “has employee ID”). First, define a policy in `Program.cs`:

```csharp
builder.Services.AddAuthorization(options =>
{
    options.AddPolicy("AtLeast21", policy =>
        policy.Requirements.Add(new MinimumAgeRequirement(21)));

    options.AddPolicy("EmployeeOnly", policy =>
        policy.RequireClaim("EmployeeId"));
});
```

Then apply it:

```csharp
[Authorize(Policy = "AtLeast21")]
public IActionResult BuyAlcohol() => View();
```

To implement a custom requirement, you need a handler (see below).

### 13.6 Role‑based and Policy‑based Authorization

#### Role‑Based Authorization

Roles are simple: you assign roles to users, and you check for them. In Identity, roles are managed with `RoleManager<IdentityRole>` and assigned via `UserManager`. You can add roles to the `ClaimsPrincipal` as role claims.

**Checking roles in code:**

```csharp
if (User.IsInRole("Admin"))
{
    // do admin stuff
}
```

#### Policy‑Based Authorization

Policies can encapsulate any logic. For example, a minimum age requirement.

1. **Create a requirement class:**

```csharp
public class MinimumAgeRequirement : IAuthorizationRequirement
{
    public int MinimumAge { get; }
    public MinimumAgeRequirement(int minimumAge) => MinimumAge = minimumAge;
}
```

2. **Create a handler for the requirement:**

```csharp
public class MinimumAgeHandler : AuthorizationHandler<MinimumAgeRequirement>
{
    protected override Task HandleRequirementAsync(AuthorizationHandlerContext context, MinimumAgeRequirement requirement)
    {
        var dateOfBirthClaim = context.User.FindFirst(c => c.Type == ClaimTypes.DateOfBirth);
        if (dateOfBirthClaim == null)
            return Task.CompletedTask;

        var dateOfBirth = Convert.ToDateTime(dateOfBirthClaim.Value);
        var age = DateTime.Today.Year - dateOfBirth.Year;
        if (dateOfBirth > DateTime.Today.AddYears(-age)) age--;

        if (age >= requirement.MinimumAge)
            context.Succeed(requirement);

        return Task.CompletedTask;
    }
}
```

3. **Register the handler in DI:**

```csharp
builder.Services.AddSingleton<IAuthorizationHandler, MinimumAgeHandler>();
```

Now the policy `"AtLeast21"` will be evaluated whenever you use `[Authorize(Policy = "AtLeast21")]`.

#### Combining Policies

You can combine multiple requirements in one policy, and the user must satisfy all of them.

#### Resource‑Based Authorization

Sometimes you need to authorize based on a specific resource (e.g., a user can edit only their own blog post). This requires manual checks in the action:

```csharp
public async Task<IActionResult> Edit(int id)
{
    var post = await _postRepository.GetByIdAsync(id);
    if (post == null) return NotFound();

    if (post.AuthorId != User.FindFirstValue(ClaimTypes.NameIdentifier))
        return Forbid();

    return View(post);
}
```

### 13.7 Authorization in Razor Views

You can conditionally display UI elements based on authorization.

- Using `User.Identity.IsAuthenticated`:

```html
@if (User.Identity.IsAuthenticated)
{
    <p>Welcome, @User.Identity.Name</p>
}
```

- Using `User.IsInRole()`:

```html
@if (User.IsInRole("Admin"))
{
    <a asp-action="ManageUsers">Manage Users</a>
}
```

- Using the authorization service via `@inject`:

```html
@inject IAuthorizationService AuthorizationService

@{
    var result = await AuthorizationService.AuthorizeAsync(User, "AtLeast21");
    if (result.Succeeded)
    {
        <p>You can buy alcohol.</p>
    }
}
```

### 13.8 Securing Web APIs with JWT

For Web APIs, JWT is the typical choice. Once JWT authentication is set up as described in 13.4, you can protect endpoints with `[Authorize]`. You can also require specific scopes or roles.

#### Getting User Information in API Controllers

```csharp
[Authorize]
[HttpGet("profile")]
public IActionResult GetProfile()
{
    var userId = User.FindFirstValue(ClaimTypes.NameIdentifier);
    var email = User.FindFirstValue(ClaimTypes.Email);
    // ...
    return Ok(new { userId, email });
}
```

#### Passing JWT Tokens in Swagger

You can configure Swagger to accept JWT tokens by adding a security definition in `AddSwaggerGen`:

```csharp
c.AddSecurityDefinition("Bearer", new OpenApiSecurityScheme
{
    In = ParameterLocation.Header,
    Description = "Please enter JWT with Bearer into field",
    Name = "Authorization",
    Type = SecuritySchemeType.ApiKey,
    Scheme = "Bearer"
});

c.AddSecurityRequirement(new OpenApiSecurityRequirement
{
    {
        new OpenApiSecurityScheme
        {
            Reference = new OpenApiReference
            {
                Type = ReferenceType.SecurityScheme,
                Id = "Bearer"
            }
        },
        new string[] {}
    }
});
```

### Summary

You’ve now learned the essential security features of ASP.NET Core:

- The **claims‑based identity** model underpins everything.
- **ASP.NET Core Identity** provides a complete user management system with registration, login, roles, and more.
- **Cookie authentication** is simple for traditional web apps.
- **JWT authentication** is ideal for APIs and SPAs.
- **Authorization** can be role‑based, policy‑based, or resource‑based, with flexible `[Authorize]` attributes and programmatic checks.
- Views can adapt based on the user’s identity and permissions.

With these tools, you can build secure applications that protect sensitive data and enforce business rules.

**Exercise:**

1. Add Identity to your existing project (or create a new one). Configure it to use your existing `AppDbContext`.
2. Add registration and login pages (you can scaffold the Identity UI).
3. Protect the `ProductsController` so only authenticated users can access it.
4. Create an `Admin` role and assign a user to it. Protect an admin‑only action (e.g., a page to manage users).
5. (If you have an API) Add JWT authentication to your API project. Implement a login endpoint that returns a JWT token, and protect at least one endpoint.
6. In a view, display different content based on whether the user is authenticated and their role.

In the next chapter, **"Middleware Pipeline,"** you’ll dive deeper into the request pipeline, learning how to create custom middleware, understand execution order, and build reusable components that cross‑cut your application.